In [1]:
def WageGrowthTracker() -> str:
    """
    Returns a link to solutions on GitHub.
    """
    return "https://www.atlantafed.org/chcs/wage-growth-tracker"

print(WageGrowthTracker())


https://www.atlantafed.org/chcs/wage-growth-tracker


In [2]:
import requests
import pandas as pd

# Download and save the Excel file
url = 'https://www.atlantafed.org/-/media/documents/datafiles/chcs/wage-growth-tracker/wage-growth-data.xlsx'
with open('wage_growth_data.xlsx', 'wb') as file:
    file.write(requests.get(url).content)

# List of sheet names on Wage Growth excel sheet
sheet_names = [
    'Education', 'Age', 'Sex', 'Occupation', 'Industry', 'Census Divisions',
    'Full-Time or Part-Time', 'Job Switcher', 'MSA or non-MSA', 
    'Average Wage Quartile', 'Paid Hourly', 'Overall 12ma', 'data_overall'
]

# Function to read sheets with different skiprows parameters
#skip first row for data_overall tab, skip 2 rows on all other tabs
def read_sheet(sheet):
    skiprows = 1 if sheet == 'data_overall' else 2
    return pd.read_excel('wage_growth_data.xlsx', sheet_name=sheet, skiprows=skiprows)

# Merge sheets
merged_df = pd.concat([read_sheet(sheet) for sheet in sheet_names], axis=1)

# Save to a CSV file
merged_df.to_csv('wageGrowth.csv', index=False)


In [3]:
# Display the Wage Growth Data Set
merged_df = pd.read_csv('wageGrowth.csv')

merged_df.head()


,Unnamed: 0,High school of less,Associates degree,Bachelors degree or higher,Overall,Unnamed: 0.1,16-24,25-54,55+,Overall.1,...,Male.1,Job Stayer.1,Job Switcher.1,Paid Hourly.1,Overall: Weighted.1,Overall: Weighted 97.1,Overall: Weekly Basis,Overall: 25/20 trimmed mean,Lower 1/2 of wage distn,Upper 1/2 of wage distn
0,1997-01-01,.,.,.,.,1997-01-01,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
1,1997-02-01,.,.,.,.,1997-02-01,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
2,1997-03-01,.,.,.,.,1997-03-01,.,.,.,.,...,4.4,4.1,5.2,4.2,4.9,4.9,4.8,4.5,4.8,4.2
3,1997-04-01,.,.,.,.,1997-04-01,.,.,.,.,...,4.6,4.1,5.4,4.3,5,5,4.9,4.5,4.9,4.2
4,1997-05-01,.,.,.,.,1997-05-01,.,.,.,.,...,4.5,4.1,5.4,4.1,4.9,5,4.8,4.4,5,4.1


In [4]:
# Employmrent rate (not in final data set right now) 
url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=LREM64TTUSM156S&scale=left&cosd=1977-01-01&coed=2024-04-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-05-22&revision_date=2024-05-22&nd=1977-01-01"

df = pd.read_csv(url)

#Rename the second column to be more clear
df = df.rename(columns={df.columns[1]: 'Employment_Rate'})
df = df.rename(columns={df.columns[0]: 'Date'})

df.to_csv('Employment_Rate.csv', index=False)

In [5]:
#Federal minimum wage and political party in power

minimumWage = pd.read_csv('MinWage_PartyControl.csv')

minimumWage_Party = minimumWage.iloc[:, :6]

print(minimumWage_Party.head())

   Year PresParty  SenParty HouseParty TrifectaFlag FedMinWage
0  1938  Democrat  Democrat   Democrat     Democrat      $0.25
1  1939  Democrat  Democrat   Democrat     Democrat      $0.30
2  1940  Democrat  Democrat   Democrat     Democrat      $0.30
3  1941  Democrat  Democrat   Democrat     Democrat      $0.30
4  1942  Democrat  Democrat   Democrat     Democrat      $0.30


In [6]:
#Federal minimum wage and political party in power

minimumWage = pd.read_csv('MinimumWage.csv')

selected_columns = minimumWage.reset_index()[['Year', 'GDP_AnnualGrowth']]

# Save the resulting DataFrame to a new CSV file
selected_columns.to_csv('annualGDP.csv', index=False)

# Print the resulting DataFrame
print(selected_columns)

    Year GDP_AnnualGrowth
0   1930           -8.50%
1   1931           -6.40%
2   1932          -12.90%
3   1933           -1.20%
4   1934           10.80%
..   ...              ...
86  2016            1.70%
87  2017            2.30%
88  2018            2.90%
89  2019            2.30%
90  2020           -3.40%

[91 rows x 2 columns]


In [7]:
merged_df = pd.merge(minimumWage_Party, selected_columns, on='Year', how='inner')

filtered_df = merged_df[merged_df['Year'] > 1997]

# Save the merged DataFrame to a new CSV file
filtered_df.to_csv('mergedMinimumWageData.csv', index=False)

# Print the merged DataFrame to verify the results
print(filtered_df.head())

    Year   PresParty    SenParty  HouseParty TrifectaFlag FedMinWage  \
60  1998    Democrat  Republican  Republican          NaN      $5.15   
61  1999    Democrat  Republican  Republican          NaN      $5.15   
62  2000    Democrat  Republican  Republican          NaN      $5.15   
63  2001  Republican    Democrat  Republican          NaN      $5.15   
64  2002  Republican    Democrat  Republican          NaN      $5.15   

   GDP_AnnualGrowth  
60            4.50%  
61            4.80%  
62            4.10%  
63            1.00%  
64            1.70%  


In [8]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup

# # URL of the webpage
# req_obj = requests.get('https://inflationdata.com/Inflation/Consumer_Price_Index/HistoricalCPI.aspx?reloaded=true#Table')

# # Parse the HTML content
# soup = BeautifulSoup(req_obj.content, 'html.parser')

# # Find all tables in the HTML
# tables = soup.find_all('table')
# table = tables[0]

# # Read the table into a pandas DataFrame
# df = pd.read_html(str(table))[0]



In [9]:

# # Create DataFrame
# df = pd.DataFrame(df)

# df_filtered = df_filtered.sort_values(by='Year')

# # Calculate month-over-month percentage change for each month
# months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# df_changes = df[['Year']].copy()

# for i in range(1, len(months)):
#     prev_month = months[i - 1]
#     current_month = months[i]
#     df_changes[current_month] = df[current_month].pct_change() * 100

# # Print the DataFrame with month-over-month changes
# print(df_changes)

# # Save the DataFrame to a CSV file
# df_changes.to_csv('CPI_Month_Over_Month_Changes.csv', index=False)

In [10]:
# year_column = df.columns[0]
# ave_column = df.columns[-1]

# # Create a new DataFrame with only the 'Year' and 'Ave' columns
# df_filtered = df[[year_column, ave_column]]

# # Rename columns for easier reference
# df_filtered.columns = ['Year', 'Ave']

# # Convert the Year column to datetime format
# df_filtered['Year'] = pd.to_datetime(df_filtered['Year'], format='%Y')


# # Sort the DataFrame by Year
# df_filtered = df_filtered.sort_values(by='Year')

# # Calculate the year-over-year percentage change based on the 'Ave' column
# df_filtered['YoY Change'] = df_filtered['Ave'].pct_change() * 100

# # Reset the index to make Year a column again
# df_filtered.reset_index(inplace=True, drop=True)

# # Display the resulting DataFrame
# print(df_filtered)

# # Save the DataFrame to a CSV file
# df_filtered.to_csv('CPI_Annual_Changes.csv', index=False)

In [11]:
#THIS DATA SET DOES NOT HAVE MONTHLY DATA ONLY ANNUAL 
# url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=USACPIBLS_FPCPITOTLZGUSA,CPIAUCSL&scale=left,left&cosd=1950-01-01,1948-01-01&coed=2022-01-01,2023-01-01&line_color=%230000ff,%23ff0000&link_values=false,false&line_style=solid,solid&mark_type=none,none&mw=3,3&lw=5,2&ost=-99999,-99999&oet=99999,99999&mma=0,0&fml=b,a&fq=Annual,Annual&fam=avg,avg&fgst=lin,lin&fgsnd=2012-01-01,2012-01-01&line_index=1,2&transformation=pc1_lin,pc1&vintage_date=2024-05-20_2024-05-20,2024-05-20&revision_date=2024-05-20_2024-05-20,2024-05-20&nd=1950-01-01_1960-01-01,1947-01-01"

# df = pd.read_csv(url)

# #Rename the second column to be more clear
# df.columns.values[1] = 'allConsumerInflation'
# df.columns.values[2] = 'UrbanConsumerInflation'

# df.to_csv('CPI.csv', index=False)

In [12]:
# # Display the CPI data sheet
# CPI_df = pd.read_csv('CPI.csv')

# CPI_df.head()

In [13]:
import pandas as pd

# URL of the dataset
url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1138&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IR3TIB01USM156N&scale=left&cosd=1964-06-01&coed=2024-04-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-05-20&revision_date=2024-05-20&nd=1964-06-01"

df = pd.read_csv(url)

#Rename the second column to be more clear
df.columns.values[1] = '3MonthInterestRate'

df.to_csv('3-Month Interest Rates', index=False)

In [14]:
# Display the 3 Month Interest Rates data sheet

InterestRates = pd.read_csv('3-Month Interest Rates')

InterestRates.head()

,DATE,3MonthInterestRate
0,1964-06-01,3.86
1,1964-07-01,3.87
2,1964-08-01,3.85
3,1964-09-01,3.87
4,1964-10-01,3.94


In [15]:

import pandas as pd
import datetime as dt
import numpy as np

# read in cpi csv
cpi = pd.read_csv('cpi.csv')

# omit first column
cpi = cpi.iloc[:, 1:]

# allows us to drop converting string to int type
cpi['Year'] = cpi['Year'].astype(int)

# sets year as index
cpi = cpi.set_index("Year")

# drops all unnecessary years and columns
cpi = cpi.drop(index = range(1913, 1996))
cpi = cpi.drop(columns = ["Avg", "Dec-Dec", "Avg-Avg"])

# 27 rows by 15 columns


# use melt function from pandas to transform cpi df
cpi_melted = cpi.reset_index().melt(id_vars='Year', var_name='Month', value_name='CPI')
month_designation = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'June': '06',
                     'July': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
cpi_melted['Month'] = cpi_melted['Month'].map(month_designation)
cpi_melted['Date'] = pd.to_datetime(cpi_melted['Year'].astype(str) + '-' + cpi_melted['Month'] + '-01')
cpi_melted = cpi_melted.drop(columns=['Year', 'Month'])
cpi_melted = cpi_melted.set_index('Date')
cpi_melted = cpi_melted.sort_values('Date')

# only include dates applicable to original df
new_cpi = cpi_melted[cpi_melted.index < '2024-04-01']

new_cpi.to_csv('new_cpi.csv')


In [16]:
#Nomrlaize CPI data above to be month over month, then creating a new datafram with only date and month over month claculations

new_cpi = pd.read_csv('new_cpi.csv')

# Calculate the month-over-month percentage change
new_cpi['MoM Change'] = new_cpi['CPI'].pct_change() * 100

print(new_cpi)

selected_columns = new_cpi.reset_index()[['Date', 'MoM Change']]

# Save the resulting DataFrame to a new CSV file
selected_columns.to_csv('CPI_Month_Over_Month_Changes.csv', index=False)

# Print the resulting DataFrame
print(selected_columns)

           Date      CPI  MoM Change
0    1996-01-01  154.400         NaN
1    1996-02-01  154.900    0.323834
2    1996-03-01  155.700    0.516462
3    1996-04-01  156.300    0.385356
4    1996-05-01  156.600    0.191939
..          ...      ...         ...
334  2023-11-01  307.051   -0.201514
335  2023-12-01  306.746   -0.099332
336  2024-01-01  308.417    0.544750
337  2024-02-01  310.326    0.618967
338  2024-03-01  312.332    0.646417

[339 rows x 3 columns]
           Date  MoM Change
0    1996-01-01         NaN
1    1996-02-01    0.323834
2    1996-03-01    0.516462
3    1996-04-01    0.385356
4    1996-05-01    0.191939
..          ...         ...
334  2023-11-01   -0.201514
335  2023-12-01   -0.099332
336  2024-01-01    0.544750
337  2024-02-01    0.618967
338  2024-03-01    0.646417

[339 rows x 2 columns]


In [17]:
import pandas as pd

# Load and filter the large DataFrame
merged_df = pd.read_csv('wageGrowth.csv', parse_dates=[0])
filtered_df = merged_df[merged_df.iloc[:, 0] > '1997-12-01']
filtered_df.columns = ['Date'] + list(filtered_df.columns[1:])  # Rename the first column

# Load and filter the 3-month interest rates DataFrame
InterestRates = pd.read_csv('3-Month Interest Rates', parse_dates=[0])
filtered_InterestRates = InterestRates[InterestRates.iloc[:, 0] > '1997-12-01']
filtered_InterestRates.columns = ['Date'] + list(filtered_InterestRates.columns[1:])  # Rename the first column

# Ensure that both 'Date' columns are in the datetime64[ns] format
filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
filtered_InterestRates['Date'] = pd.to_datetime(filtered_InterestRates['Date'])

# Merge the two filtered DataFrames on the "Date" column
merged_final_df = pd.merge(filtered_df, filtered_InterestRates, on='Date')

# Save the merged DataFrame to a new CSV file
merged_final_df.to_csv('merged_filtered_data.csv', index=False)


/tmp/ipykernel_3863/600145894.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
/tmp/ipykernel_3863/600145894.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_InterestRates['Date'] = pd.to_datetime(filtered_InterestRates['Date'])


In [18]:
# Merge MOM CPI

finalData = pd.read_csv('merged_filtered_data.csv')
for i in range(13):
    finalData = finalData.drop(columns = (f"Unnamed: 0.{(i)}"), errors = 'ignore')

# read in newcpi from previous chunk
newcpi = pd.read_csv('CPI_Month_Over_Month_Changes.csv')

# merge the two df to add cpi column
finalData = finalData.merge(newcpi, how = 'right', on = 'Date')

finalData.to_csv('merged_filtered_data.csv')

In [19]:
# Merge Employment rate 

finalData = pd.read_csv('merged_filtered_data.csv')

# read in newcpi from previous chunk
emplyRate = pd.read_csv('Employment_Rate.csv')

# merge the two df to add cpi column
finalData = finalData.merge(emplyRate, how = 'right', on = 'Date')

finalData.to_csv('finalData.csv')